In [22]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from lib import load_data
%matplotlib inline

In [23]:
users_df, costs_df, relations_df, visits_df, orders_df = load_data()

In [24]:
date = np.datetime64('2018-01-01')
date_ns = date.astype('datetime64[ns]')

In [25]:
users_df = users_df.reset_index().merge(relations_df, on=['name', 'id_partner']).set_index('id')
visits_users_df = visits_df.merge(users_df, how='left', left_on='id_user', right_index=True)

In [26]:
cur_date = np.datetime64('2018-01-20')
date_month_ago = cur_date + np.timedelta64(-4, 'W')
start_week_dates = date_month_ago + np.arange(0, 4) * np.timedelta64(1, 'W')
email_weekly_workload = pd.DataFrame({'chanel': relations_df['chanel'].unique()})
for start_week_date in start_week_dates:
    start_prev_week_date = start_week_date + np.timedelta64(-1, 'W')
    visits_mask = (visits_users_df['Visit_date'] > start_prev_week_date) & (visits_users_df['Visit_date'] < start_week_date)
    zz = visits_users_df[visits_mask].groupby(['id_user']).size()
    print(zz.head())

id_user
4618707     5
4619076    17
4619229    11
4619368     4
4619458     2
dtype: int64
id_user
4618707    7
4618807    2
4618837    2
4618940    1
4619076    8
dtype: int64
id_user
4618707     7
4618837    15
4619076     5
4619132     3
4619140     1
dtype: int64
id_user
4618613     2
4618685     1
4618707    10
4618837     7
4619076     3
dtype: int64


In [27]:

orders_slice_df = orders_df[orders_df['Order Date'] < date]
ag = orders_slice_df.groupby('id_user').agg({'Amount': 'sum'})


In [28]:
ag['sends'] = 0
ag.loc[ag['Amount'] > 5000, 'sends'] = 9
ag.loc[(ag['Amount'] >= 1000) & (ag['Amount'] < 5000), 'sends'] = 6


,Amount,sends
id_user,,
4618612,20.2224,0
4618645,8.7912,0
4618815,2.6312,0
4618982,8.7912,0
4618986,94.9960,0
4619076,8.7912,0
4619150,65.9824,0
4619480,747.9032,0
4619575,4.3912,0


In [29]:
first_visits_df = visits_df.sort_values(by='Visit_date').drop_duplicates('Visit_date', keep='first')
first_visits_earlier_date = first_visits_df[first_visits_df['Visit_date'] < date]['Visit_date']
timedelta = date_ns - pd.to_datetime(first_visits_earlier_date, unit='ns')

In [30]:
first_visits_earlier_date = first_visits_earlier_date.to_frame()

In [31]:
first_visits_earlier_date['sends'] = 0
first_visits_earlier_date.loc[timedelta < np.timedelta64(3, 'D'), 'sends'] = 3
first_visits_earlier_date.loc[(timedelta >= np.timedelta64(3, 'D')) & (timedelta < np.timedelta64(6, 'D')), 'sends'] = 4
first_visits_earlier_date.loc[timedelta > np.timedelta64(6, 'D'), 'sends'] = 5

In [73]:
visits_last_week = visits_df[(visits_df['Visit_date'] < date + np.timedelta64(7, 'D')) & (visits_df['Visit_date'] < date)]
visits_last_week_num = visits_last_week.groupby('id_user').count()
visits_last_week_num.rename(columns={'Visit_date': 'visits_num'}, inplace=True)

In [71]:
visits_last_week_num['sends'] = 1
visits_last_week_num.loc[visits_last_week_num['visits_num'] > 7, 'sends'] = 4


pandas.core.frame.DataFrame

In [72]:
visits_last_week_num

,Visit_date
id_user,
4618598,1
4618600,6
4618601,1
4618603,133
4618604,1
4618606,3
4618607,1
4618608,2
4618610,1
